In [1]:
%load_ext autoreload
%autoreload 2

import IPython
from pathlib import Path
import os
locals = IPython.extract_module_locals() # type: ignore
notebook_name = "/".join(locals[1]["__vsc_ipynb_file__"].split("/"))
os.chdir(Path(notebook_name).parent.parent)

In [2]:
import torch
from torchvision import transforms
from retail_recommender_system.data.loader import load_dataset, DataConfig, DatasetEnum
from retail_recommender_system.utils import read_imgs

In [3]:
dataset = load_dataset(DataConfig(dataset=DatasetEnum.hm, prefix="frac_0_01"))
dataset.load()

In [4]:
paths = dataset.data["items"].select("article_id_map", "path").sort(by="article_id_map").get_column("path").to_list()

In [5]:
default_size = (3, 28, 28)

def _to_rgb(x):
    return x.expand(3, -1, -1)

def _to_float(x):
    return x.to(torch.float32)

def _rescale(x):
    return (x / x.max()) * 2 - 1

transform = transforms.Compose([
    transforms.Resize(default_size[1:]),
    _to_rgb,
    _to_float,
    _rescale,
])

images = read_imgs(paths, transform, default_size=(3, 28, 28), tqdm_=True)

100%|██████████| 51941/51941 [07:37<00:00, 113.65it/s]


In [6]:
torch.save(images, ".data/hm/intermediate/frac_0_01/images.pt")